In [2]:
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim

## 1.prepare dataset

In [3]:
batch_size = 64
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))
])

train_dataset = datasets.MNIST(root='../dataset/mnist/',
                               train=True,
                               download=False, # True
                               transform=transform)
train_loader = DataLoader(train_dataset,
                          shuffle=True,
                          batch_size=batch_size)

test_dataset = datasets.MNIST(root='../dataset/mnist/',
                              train=False,
                              download=False, # 第一次下载True
                              transform=transform)
test_loader = DataLoader(test_dataset,
                         shuffle=False,
                         batch_size=batch_size)

## 2.Design Model Using Class

In [4]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.l1 = torch.nn.Linear(784,512)
        self.l2 = torch.nn.Linear(512,256)
        self.l3 = torch.nn.Linear(256,128)
        self.l4 = torch.nn.Linear(128,64)
        self.l5 = torch.nn.Linear(64,10)
    
    def forward(self,x):
        x = x.view(-1,784) # Tensor变成二阶张量即矩阵；之前的28行28列，行拼起来变成784列；第一个数是-1，将来自动算，对应的是第几个样本
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x) # 最后一层不做激活
    
model = Net()

## 3. Construct Loss and Optimizer

In [5]:
criterion = torch.nn.CrossEntropyLoss() # 交叉熵损失
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5) #带有冲量，目的能冲过鞍面和部分局部最优

## 4. Train and Test

In [7]:
def train(epoch):
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader, 0):
        inputs, target = data
        optimizer.zero_grad()
        
        # forward + backward + update
        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if batch_idx % 300 == 299:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, batch_idx + 1, running_loss / 300))
            running_loss = 0.0

In [8]:
def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, dim=1) # predicted是一行最大值的下标,dim=1表示沿着列的维度
            total += labels.size(0) # 当前batch里数据标签的数目，即batch_size
            correct += (predicted == labels).sum().item() # 张量之间的比较运算
    print('Accuracy on test set: %d %%' % (100 * correct / total))

In [9]:
if __name__ == '__main__':
    for epoch in range(10):
        train(epoch)
        test()

[1,   300] loss: 2.215
[1,   600] loss: 0.891
[1,   900] loss: 0.418
Accuracy on test set: 90 %
[2,   300] loss: 0.323
[2,   600] loss: 0.270
[2,   900] loss: 0.224
Accuracy on test set: 94 %
[3,   300] loss: 0.192
[3,   600] loss: 0.175
[3,   900] loss: 0.153
Accuracy on test set: 95 %
[4,   300] loss: 0.132
[4,   600] loss: 0.122
[4,   900] loss: 0.124
Accuracy on test set: 96 %
[5,   300] loss: 0.103
[5,   600] loss: 0.095
[5,   900] loss: 0.091
Accuracy on test set: 96 %
[6,   300] loss: 0.078
[6,   600] loss: 0.080
[6,   900] loss: 0.076
Accuracy on test set: 97 %
[7,   300] loss: 0.063
[7,   600] loss: 0.063
[7,   900] loss: 0.060
Accuracy on test set: 97 %
[8,   300] loss: 0.051
[8,   600] loss: 0.048
[8,   900] loss: 0.053
Accuracy on test set: 97 %
[9,   300] loss: 0.038
[9,   600] loss: 0.040
[9,   900] loss: 0.044
Accuracy on test set: 97 %
[10,   300] loss: 0.029
[10,   600] loss: 0.032
[10,   900] loss: 0.037
Accuracy on test set: 97 %
